In [182]:
# importing the necessary libraries
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from itertools import product
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [183]:
# creating a artificial dataset
x,y=make_classification(n_samples=10000,n_features=2,n_informative=2,n_redundant=0,n_clusters_per_class=1,random_state=68)

# splitting of train and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,random_state=42)

In [184]:
# custom defined function to split datasets

def split_fold(x_train,y_train,n_folds):
    fold_size=int(np.floor(x_train.shape[0]/n_folds))
    indexes=list(range(0,x_train.shape[0]))
    fold_indices=random.sample(list(range(0,x_train.shape[0])),fold_size)
    return fold_indices

In [191]:
# custom defined implementation of RandomSearchCV
def MyRandomizedSearchCV(param,classifier,n_iter,x_train,y_train,fold):
    # this list has n_iter number of picks
    lst=[]
    accuracies=[]
    models=[]
    
    # getting the values of the param dictionary
    param_vals=list(param.values())
    
    # getting the keys of the param dictionary
    param_keys=list(param.keys())
    
    # picking the random hyperparamteres for n_iter number of times
    for i in range(n_iter):
        x_folds=[]
        y_folds=[]
        dic={}
        
        # iterating through each hyperparameter and picking a random value
        for j in param_keys:
            dic[j]=random.choice(param[j])
            
        # appending the set of randomly picked hyper parameters in the list
        lst.append(dic)

        
        # splitting the datainto folds
        for k in range(fold):
            indices=split_fold(x_train,y_train,fold)
            x_folds.append(x_train[indices])
            y_folds.append(y_train[indices])
            
            
        
        # bulding our mode using the randomly picked set of hyper parameters 
        acc=[]
        for h in range(fold):
            
            # finding the test fold for x
            x_test_fold=x_folds[h]
                
            # removing the test fold for x
            x_train_folds=x_folds[:h]+x_folds[h+1:]
            x_train_folds_tuple=tuple(x_train_folds)
            x_train_folds_combined=np.vstack(x_train_folds_tuple)
            
            # finding the test fold for y
            y_test_fold=y_folds[h]
    
            # removing test fold for y
            y_train_folds=y_folds[:h]+y_folds[h+1:]
            y_train_folds_tuple=tuple(y_train_folds)
            y_train_folds_combined=np.hstack(y_train_folds_tuple).reshape(-1,1)
            
            # defining the train data
            for h in list(dic.keys()):
                classifier.h=dic[h]
                
            # appending this classifier to the list
            models.append(classifier)
            
            # fitting the classifier
            classifier.fit(x_train_folds_combined,y_train_folds_combined.ravel())
            
            # prediction on test data
            y_test_pred=classifier.predict(x_test_fold)
            kfold_accuracy=accuracy_score(y_test_fold,y_test_pred)
            
            # appending this score on to the acc list
            acc.append(kfold_accuracy)
            
        accuracies.append(np.array(acc).mean())
  
    # finding the best classifier with maximum accuracy
    idx_best=np.argmax(accuracies)
    
    # returning the best classifier
    return models[idx_best]

In [192]:
# calling our custom defined Randomized Search CV
param={'n_neighbors':[5,6,7],
     'leaf_size':[30,40,50]}
knn=KNeighborsClassifier()
best=MyRandomSearch(param,knn,2,x_train,y_train,5)

### MyRandomSearch(param,estimator,n_iter,x,y,fold)

##### param:
- A dictionary with keys as hyperparamter names and value as list
##### estimator
- Estimator that you want to use
##### n_iter
- no of times to randomly pick a set of hyperparameters
##### x
- array of input features
##### y
- array of target features

##### fold
- no of folds to perform KFold cross validation